In [44]:
# Import the dependencies
import pandas as pd
import pymongo
from pprint import pprint

In [47]:
# Set the column width
pd.options.display.max_colwidth = 200
# Read in the dataset. 
stations = pd.read_excel("Data/TTC Subway Stations.xlsx")
stations

,ID,Station,Category,Address,CITY,Postal Code,Line,Grade,Ridership (2019[9] avg. weekday),Parking spaces[10],...,LAT min,LAT seconds,LAT direction,LAT decimal,LON,LON degree,LON min,LON seconds,LON direction,LON decimal
0,1,Finch,TTC Subway Station,5600 Yonge Street,North York,M2N 5S2,1,Underground,102025,"3,227[13]",...,46,50.0,N,43.780556,79°24?53?W,79,24,53.0,W,79.414722
1,2,North York Centre,TTC Subway Station,5102 Yonge Street,North York,M2N 6L8,1,Underground,31471,0,...,46,6.0,N,43.768333,79°24?46?W,79,24,46.0,W,79.412778
2,3,Sheppard–Yonge,TTC Subway Station,20 Sheppard Avenue West,North York,M2N 5M9,"1,4",Underground,134076,0,...,45,41.0,N,43.761389,79°24?39?W,79,24,39.0,W,79.410833
3,4,York Mills,TTC Subway Station,4015 Yonge Street,North York,M5M 3G1,1,Underground,28461,0,...,44,39.0,N,43.744167,79°24?24?W,79,24,24.0,W,79.406667
4,5,Lawrence,TTC Subway Station,3101 Yonge Street,Toronto,M4N 3N1,1,Underground,28041,0,...,43,30.0,N,43.725000,79°24?8?W,79,24,8.0,W,79.402222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,66,Kennedy,TTC Subway Station,2455 Eglinton Avenue East,Scarborough,M1K 2P6,2,Underground,113961,729[26],...,43,57.0,N,43.732500,79°15?49?W,79,15,49.0,W,79.263611
66,67,Bayview,TTC Subway Station,2877 Bayview Ave,North York,M2K 3C8,4,Underground,8097,0,...,46,1.0,N,43.766944,79°23?12?W,79,23,12.0,W,79.386667
67,68,Bessarion,TTC Subway Station,731 Sheppard Avenue East,North York,M2K 3C9,4,Underground,3290,0,...,46,9.0,N,43.769167,79°22?35?W,79,22,35.0,W,79.376389
68,69,Leslie,TTC Subway Station,1209 Sheppard Avenue East,North York,M2K 1E2,4,Underground,5923,102[27],...,46,17.0,N,43.771389,79°21?55?W,79,21,55.0,W,79.365278


In [48]:
stations.dtypes

ID                                    int64
Station                              object
Category                             object
Address                              object
CITY                                 object
Postal Code                          object
Line                                 object
Grade                                object
Ridership (2019[9] avg. weekday)      int64
Parking spaces[10]                   object
Accessible[12]                       object
LAT                                  object
LAT degree                            int64
LAT min                               int64
LAT seconds                         float64
LAT direction                        object
LAT decimal                         float64
LON                                  object
LON degree                            int64
LON min                               int64
LON seconds                         float64
LON direction                        object
LON decimal                     

In [49]:
stations.rename(columns={'CITY': "City",
                            'Ridership (2019[9] avg. weekday)': "Ridership_Weekly_Average",
                            'Parking spaces[10]': "Parking_Spaces",
                            'Accessible[12]': "Accessibility",
                            'Postal Code': 'Postal_code',
                            'LAT': 'LAT coordinate',
                            'LON': 'LON coordinate',
                            'LAT decimal': "LAT",
                            'LON decimal': "LON"}, inplace=True)

In [50]:
stations.dtypes

ID                            int64
Station                      object
Category                     object
Address                      object
City                         object
Postal_code                  object
Line                         object
Grade                        object
Ridership_Weekly_Average      int64
Parking_Spaces               object
Accessibility                object
LAT coordinate               object
LAT degree                    int64
LAT min                       int64
LAT seconds                 float64
LAT direction                object
LAT                         float64
LON coordinate               object
LON degree                    int64
LON min                       int64
LON seconds                 float64
LON direction                object
LON                         float64
dtype: object

In [51]:
stations = stations[['Station','Category', 'Address', 'City', 'Postal_code','Line','Ridership_Weekly_Average', 'Parking_Spaces','Accessibility', 'LAT', 'LON']]
stations

,Station,Category,Address,City,Postal_code,Line,Ridership_Weekly_Average,Parking_Spaces,Accessibility,LAT,LON
0,Finch,TTC Subway Station,5600 Yonge Street,North York,M2N 5S2,1,102025,"3,227[13]",Yes,43.780556,79.414722
1,North York Centre,TTC Subway Station,5102 Yonge Street,North York,M2N 6L8,1,31471,0,Yes,43.768333,79.412778
2,Sheppard–Yonge,TTC Subway Station,20 Sheppard Avenue West,North York,M2N 5M9,"1,4",134076,0,Yes,43.761389,79.410833
3,York Mills,TTC Subway Station,4015 Yonge Street,North York,M5M 3G1,1,28461,0,Yes,43.744167,79.406667
4,Lawrence,TTC Subway Station,3101 Yonge Street,Toronto,M4N 3N1,1,28041,0,No,43.725000,79.402222
...,...,...,...,...,...,...,...,...,...,...,...
65,Kennedy,TTC Subway Station,2455 Eglinton Avenue East,Scarborough,M1K 2P6,2,113961,729[26],Yes,43.732500,79.263611
66,Bayview,TTC Subway Station,2877 Bayview Ave,North York,M2K 3C8,4,8097,0,Yes,43.766944,79.386667
67,Bessarion,TTC Subway Station,731 Sheppard Avenue East,North York,M2K 3C9,4,3290,0,Yes,43.769167,79.376389
68,Leslie,TTC Subway Station,1209 Sheppard Avenue East,North York,M2K 1E2,4,5923,102[27],Yes,43.771389,79.365278


In [46]:
conn = 'mongodb://localhost:27017'
mongo = pymongo.MongoClient(conn)
db = mongo.property_information_db